In [5]:
import numpy as np
import pandas as pd
#from WindPy import *
#w.start()
from collections import OrderedDict
from datetime import *
from  math  import *
import statsmodels.api as sm
import numpy.linalg as la   #用来做线性代数运算



In [35]:
codes =list(w.wset("sectorconstituent", "date=2018-08-01;windcode=000300.SH",usedf=True)[1].iloc[:,1])
date = '2015-01-01'
date = datetime.strptime(date,'%Y-%m-%d')
df = w.wss(','.join(codes), "ipo_issuedate",usedf=True)[1]
select_codes = df.loc[df['IPO_ISSUEDATE']<=date].index.tolist()
len(select_codes)

270

In [46]:
#估值因子
def get_values_factor(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        
        #估值因子value_factor
        factors_codes= "pe_ttm,pe_lyr,pb_lf,pb_lyr,pcf_ncf_ttm,pcf_ocf_ttm,ps_ttm,ps_lyr,val_mvtofcff" 
        factors_names=['EP_TTM','EP_LYR','BP_LF','BP_LYR','NCF_TTM','OCF_TTM','SP_TTM','SP_LYR','FCFP_LYR'] 
        factors_value=w.wss(stocks,factors_codes,"tradeDate="+date) 
        factors_value=pd.DataFrame(factors_value.Data,index=factors_names,columns=factors_value.Codes).T
        factors_value=1/factors_value
            
  
        
        #获取企业价值倍数
        factors_value['EV/EBITDA']=w.wss(stocks, "ev2_to_ebitda","tradeDate="+date).Data[0]
        
        #获取股息率
        factors_value['DYR']=w.wss(stocks, "dividendyield2","tradeDate="+date).Data[0]
        
        dict_df[date]=factors_value
    factors_values=pd.concat(dict_df.values(),keys=dict_df.keys())
    return factors_values
#规模因子
def get_size_factor(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        size_factors=w.wss(stocks, "val_lnmv,val_lnfloatmv,val_lntotassets","tradeDate="+date)
        factors_names=['LN_MV','LN_FLOAT_MV','LN_TOTAL_ASSETS']                       
        size_factors=pd.DataFrame(size_factors.Data,index=factors_names,columns=size_factors.Codes).T
        dict_df[date]=size_factors.iloc[:,:]
        #print(dict_df.values())
        #print(dict_df.keys())
    size_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return size_factors

#杠杆因子
def get_leverage_factors(dates,stocks,factors_codes,factors_names):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        leverage_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        leverage_factors=pd.DataFrame(leverage_factors.Data,index=factors_names,columns=leverage_factors.Codes).T
        dict_df[date]=leverage_factors
    leverage_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return leverage_factors
#技术因子
def get_Technical_factors(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        factors_codes="tech_rvi,tech_rstr12,tech_cyf,tech_cry,tech_cr20"
        factors_names=['RVI','RSTR12','CYF','CRY','CR20']
        Technical_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        Technical_factors=pd.DataFrame(Technical_factors.Data,index=factors_names,columns=Technical_factors.Codes).T
         #获取RSI指标
        Technical_factors['RSI']=w.wss(stocks, "RSI","tradeDate="+date+";RSI_N=6;priceAdj=F;cycle=D").Data[0] 
        #获取DEA异同平均数指标
        Technical_factors['DEA']=w.wss(stocks, "MACD","tradeDate="+date+";MACD_L=26;MACD_S=12;MACD_N=9;MACD_IO=2;priceAdj=F;cycle=D").Data[0]
        #获取MACD指标
        Technical_factors['MACD']=w.wss(stocks, "MACD","tradeDate="+date+";MACD_L=26;MACD_S=12;MACD_N=9;MACD_IO=3;priceAdj=F;cycle=D").Data[0]
        #获取K\D\J
        Technical_factors['K']=w.wss(stocks, "KDJ","tradeDate="+date+";KDJ_N=9;KDJ_M1=3;KDJ_M2=3;KDJ_IO=1;priceAdj=F;cycle=D").Data[0]
        Technical_factors['D']=w.wss(stocks, "KDJ","tradeDate="+date+";KDJ_N=9;KDJ_M1=3;KDJ_M2=3;KDJ_IO=2;priceAdj=F;cycle=D").Data[0]
        Technical_factors['J']=w.wss(stocks, "KDJ","tradeDate="+date+";KDJ_N=9;KDJ_M1=3;KDJ_M2=3;KDJ_IO=3;priceAdj=F;cycle=D").Data[0]
        
        dict_df[date]=Technical_factors
    Liquidation_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return Liquidation_factors
#动量因子
def get_Momentum_factors(dates,stocks):
    dict_df=OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        factors_codes="tech_revs5,tech_revs10,tech_revs60,tech_revs120,tech_revs250,tech_revs750,tech_revs1mmax,tech_lnhighlow20d"
        factors_names=['REV_5D','REV_10D','REV_3M','REV_6M','REV_1Y','REV_3Y','REV_LAST1M_MAX','LN_HIGH-LOW']
        Momentum_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        Momentum_factors=pd.DataFrame(Momentum_factors.Data,index=factors_names,columns=Momentum_factors.Codes).T
        dict_df[date]=Momentum_factors
    Momentum_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return Momentum_factors

#获取成长因子
def get_growth_factors(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        factors_codes= "fa_orgr_ttm,fa_nagr,fa_gpmgr_ttm,fa_npgr_ttm,fa_tagr,fa_ncgr_ttm,fa_cfigr_ttm,fa_cffgr_ttm,fa_cfogr_ttm,fa_oigr_ttm" 
        factors_names=['sales_gr_TTM','net_asset_gr_TTM','gross_margin_gr_TTM','net_profit_gr_TTM','total_asset_gr_TTM','net_cash_flow_gr_TTM','invest_cash_flow_gr_TTM','finance_cash_folw_gr_TTM','operate_cash_flow_gr_TTM','operete_profit_gr_TTM']
        growth_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        growth_factors=pd.DataFrame(growth_factors.Data,index=factors_names,columns=growth_factors.Codes).T
        #growth_factors['eps_growth_TTM']=w.wss(A_stocks, "yoyeps_basic","rptDate="+date+";N=1").Data[0]  #基本每股收益同比增长率
        #growth_factors['roe_growth_TTM']=w.wss(A_stocks, "growth_roe","rptDate="+date+";N=1").Data[0]  #净资产收益率N年同比增长率
        dict_df[date]=growth_factors
        growth_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return growth_factors
#市值因子
def get_assisted_factors(dates,stocks):   
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        assisted_factors=w.wss(stocks, "industry_sw,mkt_cap_ashare","tradeDate="+date+';industryType=1;unit=1')
        factors_names=['INDUSTRY_SW','CAP']                       
        assisted_factors=pd.DataFrame(assisted_factors.Data,index=factors_names,columns=assisted_factors.Codes).T
        dict_df[date]=assisted_factors
    assisted_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return assisted_factors
#获取每月交易日期序列
def get_trade_date(start_date, end_date, period='M'):
    data = w.tdays(start_date, end_date, period=period) #获取每月最后一个交易日
    trade_dates = data.Data[0]
    trade_dates = [dt.strftime("%Y-%m-%d") for dt in trade_dates]
    return trade_dates
def get_feature_names(data):  #该函数用于获取数据集中需测试的因子名
    columns = data.columns.tolist()
    fea_names = [i for i in columns if i not in ["INDUSTRY_SW",'CAP'] ]
    return fea_names
def extreme_process_MAD(Data):
    feature_names = get_feature_names(Data)
    median=Data[feature_names].median(axis=0)  #获取中位数
    MAD=abs(Data[feature_names].sub(median,axis=1)).median(axis=0)
    for j in range(len(MAD)):
        for i in range(Data.shape[0]):
            if np.isnan(Data.iloc[i,j]) == False:
                if Data.iloc[i,j] <= median[j]-5*1.4826*MAD[j]:
                    Data.iloc[i,j] = median[j]-5*1.4826*MAD[j]
                if Data.iloc[i,j] >= median[j]+5*1.4826*MAD[j]:
                    Data.iloc[i,j] = median[j]+5*1.4826*MAD[j]
                    
    return Data
        
        
def fill_missing_value(Data):
    feature_names = get_feature_names(Data)
    for j in range(len(feature_names)):
        industry_fill_value = Data[feature_names[j]].groupby(Data['INDUSTRY_SW']).mean()
        #print(j,list(industry_fill_value))
        for i in range(Data.shape[0]):
            #if i < 3:
                #print(Data.iloc[i,:]['INDUSTRY_SW'])
            if np.isnan(Data.iloc[i,j]):
                #print(industry_fill_value[Data.iloc[i,-2]])
                Data.iloc[i,j] = industry_fill_value[Data.iloc[i,-2]]
                #print(Data.iloc[i,j])
                #print(np.isnan(Data.iloc[i,j]))
                break
    return Data
def data_scale_CAP(data):
    feature_names = get_feature_names(data)
    data_=data.copy()
    cap_weight = data_["CAP"]/ data_["CAP"].sum()
    for name in feature_names:
        avg=(data_[name]*cap_weight).sum()
        data_[name]=(data_[name]-avg)/data_[name].std()
    return data_

def data_scale_neutral(data):
    feature_names = get_feature_names(data)
    data_=data.copy()
    industrys=data['INDUSTRY_SW']  #获取所属申万一级行业代码
    data_med = pd.get_dummies(data,columns=['INDUSTRY_SW'],drop_first=True)
    n = len(data['INDUSTRY_SW'].unique())    #确定产生虚拟变量个数
    X = np.array(data_med[data_med.columns[-(n-1):]])  #行业虚拟变量作为为自变量
    for name in feature_names:
        y = np.array(data_[name])
        if la.matrix_rank(X.T.dot(X)) == (n-1): #当矩阵满秩时，估计回归参数
            beta_ols = la.inv(X.T.dot(X)).dot(X.T).dot(y)  
            residual = y - X.dot(beta_ols)      #计算残差，并将其作为剔除行业影响的因子值 
        else:
            residual = y   #如果逆不存在的话 则 用原值
        data_[name]=residual
    return data_
    return data_
#因变量涨跌幅的获取以及处理
def get_pct(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        h = "tradeDate="+date+";cycle=M"
        factors_value=w.wss(stocks,"pct_chg",h,usedf=True)[1]
        dict_df[date]=factors_value
    d=pd.concat(dict_df.values(),keys=dict_df.keys())
    return d
def accuracy(data1,data2):
    n=0
    for i in range(len(data1)):
        if data1[i] == data2[i]:
            n+=1
    acc = n/len(data1)
    return acc
def one_hot(data):
    DataY = pd.DataFrame(data)
    l = []
    for i in range(DataY.shape[1]):
        for j in range(DataY.shape[0]):
            if DataY.iloc[j,i]==0:
                l.append([1,0,0])
            elif DataY.iloc[j,i]==1:
                l.append([0,1,0])
            else:
                l.append([0,0,1])
    return l
  


In [37]:
start_date='20150101'
end_date='20160630'
dates=get_trade_date(start_date, end_date, period='M')

In [38]:
start_date='20150101'
end_date='20160630'
dates=get_trade_date(start_date, end_date, period='M')
values_factor=get_values_factor(dates,select_codes)
size_factor=get_size_factor(dates,select_codes)
factors_codes="fa_current,fa_quick,fa_blev,fa_debttoasset,fa_cfotocurliabs_ttm,fa_debttoequity"
factors_names=['CUR','QR','BOOK_LEVEL','DEBT_TO_ASSETS','CASH_FLOW_LIABILITY','DEBT_TO_EQUITY']
leverage_factors = get_leverage_factors(dates,select_codes,factors_codes,factors_names)
Technical_factors = get_Technical_factors(dates,select_codes)
Momentum_factors = get_Momentum_factors(dates,select_codes)
assisted_factors = get_assisted_factors(dates,select_codes)
growth_factors=get_growth_factors(dates,select_codes)
Data= pd.concat([values_factor,growth_factors,leverage_factors,Momentum_factors,Technical_factors,assisted_factors],axis=1)


In [39]:
print(Data)

                        EP_TTM    EP_LYR     BP_LF    BP_LYR   NCF_TTM  \
2015-01-30 000001.SZ  0.120824  0.095703  0.796337  0.822809  0.090356   
           000002.SZ  0.106428  0.104372  0.608822  0.608649  0.081991   
           000060.SZ  0.030768  0.019462  0.297739  0.284096  0.007500   
           000063.SZ  0.039983  0.020592  0.376944  0.377337 -0.019092   
           000069.SZ  0.075357  0.076936  0.456795  0.490742  0.028738   
...                        ...       ...       ...       ...       ...   
2016-05-31 601991.SH  0.050907  0.052630  0.855959  0.848693 -0.081920   
           601992.SH  0.047457  0.047353  0.897671  0.733268  0.216552   
           601998.SH  0.145972  0.145014  1.157839  1.119505  0.266688   
           603288.SH  0.032021  0.031077  0.098463  0.108367 -0.000878   
           603993.SH  0.009170  0.011677  0.269613  0.266221  0.103409   

                       OCF_TTM    SP_TTM    SP_LYR  FCFP_LYR  EV/EBITDA  ...  \
2015-01-30 000001.SZ -0.089589 

In [40]:
Data.to_csv('data.csv')
Data = pd.read_csv('data.csv',index_col=[0,1])

In [41]:


Data.describe()

,EP_TTM,EP_LYR,BP_LF,BP_LYR,NCF_TTM,OCF_TTM,SP_TTM,SP_LYR,FCFP_LYR,EV/EBITDA,...,CYF,CRY,CR20,RSI,DEA,MACD,K,D,J,CAP
count,4551.000000,4551.000000,4551.000000,4515.000000,4551.000000,4551.000000,4551.000000,4551.000000,4551.000000,3879.000000,...,4187.000000,4.281000e+03,4183.000000,4.502000e+03,4551.000000,4551.000000,4551.000000,4551.000000,4551.000000,4.551000e+03
mean,0.039140,0.036990,0.386266,0.370687,0.055076,0.103791,0.600126,0.581380,-0.005738,39.283001,...,59.245151,7.292720e+24,1.192691,4.868801e+01,0.055841,-0.061262,46.027759,48.681234,40.720810,7.535709e+10
std,0.073957,0.061976,0.285863,0.290615,0.204830,0.282612,0.749592,0.744014,0.233031,179.379262,...,20.855924,2.028301e+24,3.765446,1.754812e+01,0.793883,0.570622,21.951729,20.190894,32.665830,1.578043e+11
min,-1.668133,-1.661740,-1.619825,-1.595841,-0.781347,-0.795183,0.000468,0.000520,-2.751632,-1987.456064,...,1.150171,4.468895e+01,-25.727364,2.376655e-10,-6.532764,-7.759711,0.031167,0.219398,-29.586237,1.016622e+09
25%,0.014614,0.013200,0.174325,0.159644,-0.007708,0.009218,0.141322,0.114457,-0.022796,10.636482,...,44.695860,7.600975e+24,0.655206,3.536313e+01,-0.199496,-0.150255,27.777133,32.097972,13.937637,2.134529e+10
50%,0.031386,0.027553,0.305787,0.291202,0.007757,0.035481,0.354326,0.334027,0.005326,18.519086,...,62.681517,7.866346e+24,0.939094,4.668013e+01,0.015609,-0.000498,43.675095,49.248463,35.849651,3.507614e+10
75%,0.061806,0.054067,0.531413,0.516968,0.045703,0.102480,0.721174,0.729815,0.047075,32.879641,...,76.282959,8.046438e+24,1.414323,6.079135e+01,0.263460,0.102152,63.760078,64.732888,66.250720,6.531703e+10
max,0.202277,0.201166,1.682174,1.549575,2.675450,4.321713,7.758417,6.768021,0.915687,3977.482834,...,96.288252,8.896789e+24,201.938462,1.000000e+02,11.619822,4.894894,99.965152,99.746595,127.101726,2.168137e+12


In [47]:
Data2 =Data.groupby(level=0).apply(extreme_process_MAD)
Data2.to_csv('svm_data_2.csv')
Data2 = pd.read_csv('svm_data_2.csv',index_col=[0,1])
Data3=fill_missing_value(Data2)

In [50]:
Data3.head(20)

EP_TTM    EP_LYR     BP_LF    BP_LYR   NCF_TTM  \
2015-01-30 000001.SZ  0.120824  0.095703  0.796337  0.822809  0.090356   
           000002.SZ  0.106428  0.104372  0.608822  0.608649  0.081991   
           000060.SZ  0.030768  0.019462  0.297739  0.284096  0.007500   
           000063.SZ  0.039983  0.020592  0.376944  0.377337 -0.019092   
           000069.SZ  0.075357  0.076936  0.456795  0.490742  0.028738   
           000100.SZ  0.083312  0.057359  0.478857  0.494817 -0.077497   
           000157.SZ  0.022144  0.081402  0.878392  0.864941 -0.052101   
           000166.SZ  0.007471  0.007471  0.140309  0.085987 -0.017667   
           000333.SZ  0.080982  0.042002  0.300851  0.311772 -0.089317   
           000338.SZ  0.101242  0.068064  0.588346  0.618882  0.097771   
           000402.SZ  0.070091  0.095217  0.719661  0.797467 -0.141386   
           000413.SZ  0.035540  0.017297  0.288627  0.301488 -0.026465   
           000415.SZ  0.057745  0.045008  0.412413  0.418575 -0.099310   
           000423.SZ  0.050560  0.048059  0.219199  0.236620 -0.011512   
           000425.SZ  0.047784  0.055889  0.799141  0.423224 -0.060823   
           000503.SZ  0.000405  0.000394  0.038496  0.039786  0.004951   
           000538.SZ  0.038251  0.035417  0.162120  0.170645 -0.012205   
           000540.SZ  0.067991  0.044374  0.286578  0.299383 -0.033502   
           000559.SZ  0.031159  0.022079  0.157508  0.165611 -0.049391   
           000568.SZ  0.074021  0.128224  0.375603  0.362280 -0.012497   

                       OCF_TTM    SP_TTM    SP_LYR  FCFP_LYR  EV/EBITDA  ...  \
2015-01-30 000001.SZ -0.089589  0.436667  0.327926  0.100122        NaN  ...   
           000002.SZ  0.215316  0.932968  0.934872  0.256925   5.732072  ...   
           000060.SZ  0.050561  1.348924  0.994821  0.006285  16.856011  ...   
           000063.SZ  0.102375  1.232218  1.141080 -0.064829  12.206895  ...   
           000069.SZ -0.046700  0.525569  0.491401 -0.040159   7.123298  ...   
           000100.SZ  0.141872  2.499047  2.320488 -0.090326   5.330813  ...   
           000157.SZ -0.094596  0.623633  0.817248 -0.274822  25.672277  ...   
           000166.SZ -0.025663  0.023861  0.023861 -0.073947        NaN  ...   
           000333.SZ  0.145981  1.076711  0.955563  0.009574   7.438602  ...   
           000338.SZ  0.173906  1.355754  1.111506  0.155011   4.447374  ...   
           000402.SZ -0.132876  0.632444  0.654731 -0.211237  12.169386  ...   
           000413.SZ -0.087086  0.074090  0.043649 -0.031072  18.607320  ...   
           000415.SZ  0.266650  0.393556  0.272671 -0.289855  16.875361  ...   
           000423.SZ  0.026914  0.150578  0.160463  0.023458  13.617204  ...   
           000425.SZ -0.006605  0.957887  1.000129  0.058973  19.028455  ...   
           000503.SZ  0.003828  0.005832  0.006252  0.002691  69.151856  ...   
           000538.SZ  0.019053  0.273401  0.241279 -0.010583  21.332364  ...   
           000540.SZ  0.026919  0.430639  0.310928  0.008307  13.609526  ...   
           000559.SZ  0.067794  0.418610  0.385397  0.092989  17.206326  ...   
           000568.SZ  0.060654  0.262901  0.389060  0.048458  17.298804  ...   

                               CRY      CR20        RSI       DEA      MACD  \
2015-01-30 000001.SZ  8.319482e+24  0.584092  35.086174  0.043812 -0.202159   
           000002.SZ  8.248641e+24  0.913342  47.638434  0.306137 -0.183652   
           000060.SZ  8.152275e+24  1.404596  53.744557  0.109279  0.013691   
           000063.SZ  8.177552e+24  1.464124  41.614252  0.427419 -0.182422   
           000069.SZ  6.809201e+24  1.060832  58.209361  0.176799 -0.045605   
           000100.SZ  8.194097e+24  1.213904  49.620844  0.041733 -0.038007   
           000157.SZ  8.373075e+24  0.412946  37.784767  0.010519 -0.091570   
           000166.SZ  6.809201e+24       NaN  45.878926 -0.156901 -0.419610   
           000333.SZ  7.881851e+24  1.408541  40.252108  0.606635 -0.431248  

In [51]:
industry_fill_value = Data['CAP'].groupby(Data['INDUSTRY_SW']).mean()

In [53]:
for i in range(Data3.shape[0]):
    if np.isnan(Data3.iloc[i,-1]):
        Data3.iloc[i,-1] = industry_fill_value[Data3.iloc[i,:]['INDUSTRY_SW']]
Data3 = Data3.fillna(Data3.mean())
Data4 = data_scale_CAP(Data3)
Data5 = data_scale_neutral(Data4)
Data6 = Data5.drop(['INDUSTRY_SW'],axis=1)
mean = Data6.mean()
std = Data6.std()
Data7 = (Data6-mean)/std
Data7.to_csv('SVM_X.csv')

In [54]:
pct_start_date='20150201'
pct_end_date='20160731'
pct_dates=get_trade_date(pct_start_date,pct_end_date, period='M')
pct=get_pct(pct_dates,select_codes)
y = pct.dropna()#去除缺失的涨跌幅数据。
y = y.sort_values(by='PCT_CHG')

In [82]:
y

PCT_CHG
2016-01-29 002153.SZ  -46.960265
2015-07-31 600061.SH  -45.478583
2015-08-31 600376.SH  -41.907357
2016-01-29 002411.SZ  -41.895262
           002460.SZ  -41.554108
...                          ...
2015-04-30 300059.SZ  185.350102
2015-12-31 002468.SZ  185.412108
2016-06-30 002352.SZ  217.150396
2015-11-30 002558.SZ  279.778831
2015-04-30 002625.SZ  314.739884

[4553 rows x 1 columns]

In [55]:
train_y = pd.concat([y.head(2000),y.tail(2000)])
train_y2 = train_y.reset_index()

In [59]:
train_y.head(50)

PCT_CHG
2016-01-29 002153.SZ -46.960265
2015-07-31 600061.SH -45.478583
2015-08-31 600376.SH -41.907357
2016-01-29 002411.SZ -41.895262
           002460.SZ -41.554108
           600588.SH -41.433511
2015-07-31 002044.SZ -41.363636
2015-08-31 300033.SZ -41.260826
2015-07-31 600570.SH -41.178046
           000728.SZ -40.731964
2015-09-30 600886.SH -40.551446
2016-01-29 601216.SH -40.436681
2015-06-30 600704.SH -40.411110
2016-01-29 601155.SH -39.931068
           002602.SZ -39.911894
           601866.SH -39.630682
2015-06-30 300408.SZ -39.604990
2016-01-29 000559.SZ -39.505082
           601919.SH -39.467849
           600346.SH -38.769231
           002050.SZ -38.591549
2015-06-30 600820.SH -38.263666
2016-01-29 000839.SZ -37.936118
2015-06-30 601766.SH -37.657046
2016-01-29 600100.SH -37.334071
2015-09-30 000959.SZ -37.296417
2015-07-31 002625.SZ -37.235709
2016-01-29 300122.SZ -37.108301
2015-06-30 600637.SH -36.702768
2015-08-31 000709.SZ -36.571429
2016-01-29 600705.SH -36.020911
           601618.SH -35.880399
           000060.SZ -35.495367
           600415.SH -35.473341
2015-07-31 600739.SH -35.255624
2016-01-29 002352.SZ -34.989066
           300027.SZ -34.956606
           600085.SH -34.835239
           600487.SH -34.755332
           601718.SH -34.612031
           600570.SH -34.607184
           600482.SH -34.495218
           600050.SH -34.466019
2015-07-31 600809.SH -34.454674
2015-06-30 002050.SZ -34.380354
2016-01-29 600339.SH -34.308511
           601989.SH -34.148936
2015-08-31 601111.SH -34.127660
2016-01-29 002074.SZ -34.070081
2015-08-31 600588.SH -33.929027

In [60]:
pct_dates.insert(0,dates[0])#为了让日期索引对齐加入第一个月份
#将每个月份前退一个月的交易日
for i in range(len(train_y2.iloc[:,0])):
    train_y2.iloc[i,0]=pct_dates[pct_dates.index(train_y2.iloc[i,0])-1]
train_y3 = train_y2.set_index(['level_0','level_1'])
Y = train_y3#采用的是月的涨跌幅所以会比较大

In [71]:
X = pd.read_csv('SVM_X.csv',index_col=[0,1])
X = X.loc[Y.index]
Y.iloc[:2000,0]=0
Y.iloc[2000:,0]=1


In [72]:
pd.set_option('display.max_rows', 1000) # 最多展示1000行
pd.set_option('display.max_columns', 50) # 最多展示10列
X.head(10)

EP_TTM    EP_LYR     BP_LF    BP_LYR   NCF_TTM  \
level_0    level_1                                                       
2015-12-31 002153.SZ -0.787259 -0.782339 -1.076258 -1.002419 -0.276304   
2015-06-30 600061.SH -0.844098 -0.956575 -0.778008 -1.289807  1.100242   
2015-07-31 600376.SH  0.123302  0.029276 -0.109141 -0.059637 -0.341009   
2015-12-31 002411.SZ -0.945078 -0.929839 -0.264907 -0.581224 -0.354969   
           002460.SZ -0.846121 -0.868852 -1.101062 -1.016840 -0.402135   
           600588.SH -0.848407 -0.665632 -1.007394 -0.887465 -0.116881   
2015-06-30 002044.SZ -0.879881 -0.877892 -1.283871 -1.216356 -0.489792   
2015-07-31 300033.SZ -0.878294 -0.918324 -1.254323 -1.194753 -0.237841   
2015-06-30 600570.SH -0.828608 -0.829336 -1.271947 -1.208078 -0.316883   
           000728.SZ -0.424782 -0.504535 -0.521830 -0.482458  1.493565   

                       OCF_TTM    SP_TTM    SP_LYR  FCFP_LYR     EV/EBITDA  \
level_0    level_1                                                           
2015-12-31 002153.SZ -0.574561 -0.878713 -0.846270 -0.181754  2.544187e+00   
2015-06-30 600061.SH -0.397266 -0.887955 -0.858012 -0.077905  3.537373e+00   
2015-07-31 600376.SH -0.285822 -0.039722 -0.059038 -0.161324 -1.338195e-01   
2015-12-31 002411.SZ -0.661100 -0.880329 -0.843815 -0.374430  4.183384e-01   
           002460.SZ -0.595299 -0.864545 -0.853290 -0.013347  2.544187e+00   
           600588.SH -0.548406 -0.789173 -0.755910 -0.250235  2.247915e+00   
2015-06-30 002044.SZ -0.584184 -0.864288 -0.838169 -0.055895  3.537373e+00   
2015-07-31 300033.SZ -0.549827 -0.927724 -0.903186 -0.047059  3.337955e+00   
2015-06-30 600570.SH -0.522167 -0.906610 -0.880343 -0.058916  3.537373e+00   
           000728.SZ  0.581202 -0.849134 -0.836104 -0.855701 -3.858025e-15   

                               DYR  sales_gr_TTM  net_asset_gr_TTM  \
level_0    level_1                                                   
2015-12-31 002153.SZ -1.024287e+00     -0.605971         -0.119159   
2015-06-30 600061.SH -4.329870e-15     -0.390570          2.278918   
2015-07-31 600376.SH -2.442029e-02      0.601089         -0.541566   
2015-12-31 002411.SZ -1.024146e+00     -0.720673         -0.911065   
           002460.SZ -9.590693e-01      0.326977          0.263006   
           600588.SH -4.691416e-01     -0.678906          0.778748   
2015-06-30 002044.SZ -9.853695e-01     -0.587175         -0.468021   
2015-07-31 300033.SZ -1.034206e+00      1.408990         -0.489564   
2015-06-30 600570.SH -9.503888e-01     -0.053850         -0.181453   
           000728.SZ -8.675354e-01      1.328643         -0.350987   

                      gross_margin_gr_TTM  net_profit_gr_TTM  \
level_0    level_1                                             
2015-12-31 002153.SZ         3.945267e-01          -0.369581   
2015-06-30 600061.SH        -1.365193e+00           1.817123   
2015-07-31 600376.SH        -2.223930e-01          -0.064459   
2015-12-31 002411.SZ         1.837448e+00           2.160911   
           002460.SZ        -4.292115e-01           0.116593   
           600588.SH         1.181954e+00          -0.979162   
2015-06-30 002044.SZ         1.643544e-01           1.817123   
2015-07-31 300033.SZ         5.436387e-02           1.817123   
2015-06-30 600570.SH         5.187625e-01          -0.291497   
           000728.SZ         4.302114e-16           0.710461   

                      total_asset_gr_TTM  net_cash_flow_gr_TTM  \
level_0    level_1                                               
2015-12-31 002153.SZ           -0.268776             -0.031062   
2015-06-30 600061.SH            2.097165              2.101444   
2015-07-31 600376.SH           -0.597455             -0.172678   
2015-12-31 002411.SZ            0.024333             -0.194204   
           002460.SZ            0.578233             -0.619826   
           600588.SH            0.105730             -0.320822   
2015-06-30 002044.SZ           -1.613825             -0.7687

In [2]:
from sklearn.svm import SVC,SVR
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.svm import SVC
poly3clf = SVC(C=0.01,kernel='poly',degree=3, gamma=0.3,probability=True).fit(X,Y)
poly7clf = SVC(C=0.1,kernel='poly',degree=7,gamma=0.1,probability=True).fit(X,Y)
linear = SVC(C=3,kernel='linear',probability=True).fit(X,Y)
sigmoid = SVC(C=10,kernel='sigmoid', gamma=0.001,probability=True).fit(X,Y)
rbf = SVC(C=3,kernel='rbf',degree=3,gamma=0.03,probability=True).fit(X,Y)

NameError: name 'X' is not defined

In [ ]:
test_y= y[2000:-2000]
test_y.index
test_x =X.loc[test_y.index].dropna()
poly3_pre = poly3clf.predict(test_x)
poly7_pre =  poly7clf.predict(test_x)
linear_pre = linear.predict(test_x)
sigmoid_pre = sigmoid.predict(test_x)
rbf_pre = rbf.predict(test_x)
real_y = test_y.loc[test_x.index]

In [4]:
test_y

NameError: name 'test_y' is not defined

In [79]:
test_x

NameError: name 'test_x' is not defined